In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm 
from tqdm import trange
import time
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.DataFrame(columns=['INDEX','LINK','LINK_RTF'])

In [4]:
root_url = 'https://www.corteconstitucional.gov.co'

In [5]:
for i in tqdm(range(306,358)):
    
    try:
        page_url = '/relatoria/providencia.aspx?pg='+ str(i) +'&buscar=&anio=&total=35718'
        main_url = root_url + page_url

        main_html_text = requests.get(main_url).text
        main_soup = BeautifulSoup(main_html_text, 'lxml')

        docs = main_soup.find_all('div', class_ = 'grow')

        for doc in docs:
            
            
            doc_link = doc.find('a')['href']

            inner_doc_url = root_url + doc_link
            
            try:
                
                html_inner_doc = requests.get(inner_doc_url).text
                inner_doc_soup = BeautifulSoup(html_inner_doc, 'lxml')
                doc_index = doc.find('a').text
                doc_rtf_link = root_url + (inner_doc_soup.find('a')['href'])

                new_row = {'INDEX':doc_index,'LINK':inner_doc_url,'LINK_RTF':doc_rtf_link}
                data = data.append(new_row, ignore_index=True)
            
            except Exception as e:
                
                print("Error!: " + str(e))
                print('Problema en: '+ inner_doc_url)
                print('Pagina: ' + main_url)
                print('Etapa:' + str(i))
                print('*'*30)
            

    except Exception as e:
        print("Error!: " + str(e))

100%|██████████| 52/52 [1:25:59<00:00, 99.23s/it] 


In [6]:
data

,INDEX,LINK,LINK_RTF
0,T-898-99,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
1,T-889-99,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
2,T-888-99,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
3,T-887-99,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
4,T-886-99,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
...,...,...,...
5214,C-416-92,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5215,C-221-92,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5216,C-005-92,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5217,C-004-92,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...


In [7]:
file_name = '/content/drive/My Drive/LINKS_7.csv'
data.to_csv(file_name, encoding='utf-8', index= False)